In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

In [2]:
!pip install torch

In [4]:
dataset = ''
with open('Merchant of Venice.txt', 'r') as f:
    # Read the file line by line
    for line in f:
        # Print or process each line
        if line[0] == ' ':
            line = line.lower()
            line = line.strip()
            line = line + '\n'
            line_chars = [char for char in line]
            dataset += line

In [5]:
chars = sorted(list(set(''.join(dataset))))
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
print(itos)
print(stoi)

{0: '\n', 1: ' ', 2: '!', 3: "'", 4: ',', 5: '-', 6: '.', 7: ':', 8: ';', 9: '?', 10: '[', 11: ']', 12: 'a', 13: 'b', 14: 'c', 15: 'd', 16: 'e', 17: 'f', 18: 'g', 19: 'h', 20: 'i', 21: 'j', 22: 'k', 23: 'l', 24: 'm', 25: 'n', 26: 'o', 27: 'p', 28: 'q', 29: 'r', 30: 's', 31: 't', 32: 'u', 33: 'v', 34: 'w', 35: 'x', 36: 'y', 37: 'z'}
{'\n': 0, ' ': 1, '!': 2, "'": 3, ',': 4, '-': 5, '.': 6, ':': 7, ';': 8, '?': 9, '[': 10, ']': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37}


In [6]:
class NextChar(nn.Module):
  def __init__(self, context, vocab_size, emb_dim, hidden_size):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, emb_dim)
    self.lin1 = nn.Linear(context * emb_dim, hidden_size)
    self.lin2 = nn.Linear(hidden_size, vocab_size)

  def forward(self, x):
    x = self.emb(x)
    x = x.view(x.shape[0], -1)
    x = torch.sin(self.lin1(x))
    x = self.lin2(x)
    return x

In [17]:
context = 0

In [ ]:
X, Y = [], []
for i in range(context, len(dataset)):
    x = [stoi[dataset[i-k]] for k in range(context, 0, -1)]
    X.append(x)
    Y.append(stoi[dataset[i]])

In [ ]:
X = torch.tensor(X)
Y = torch.tensor(Y)

In [ ]:
model = NextChar(context, len(stoi), emb_size, hidden_layer_size)

In [ ]:
# Train the model

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.01)
import time
# Mini-batch training
batch_size = 4096
print_every = 100
elapsed_time = []
for epoch in range(1000):
    start_time = time.time()
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i+batch_size]
        y = Y[i:i+batch_size]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
    end_time = time.time()
    elapsed_time.append(end_time - start_time)
    if epoch % print_every == 0:
        print(epoch, loss.item())

In [ ]:
torch.save(model.state_dict(), 'trained_models/{}_{}_{}'.format(context, emb_size, hidden_layer_size))

In [18]:
for i in [2,3,4,5,6]:
    print("emb_size", i)
    emb_size = i
    for j in [5,10,15,20]:
        print("\thidden_layer_size", j)
        hidden_layer_size = j
        X, Y = [], []
        for i in range(context, len(dataset)):
            x = [stoi[dataset[i-k]] for k in range(context, 0, -1)]
            X.append(x)
            Y.append(stoi[dataset[i]])
        X = torch.tensor(X)
        Y = torch.tensor(Y)
        model = NextChar(context, len(stoi), emb_size, hidden_layer_size)
        loss_fn = nn.CrossEntropyLoss()
        opt = torch.optim.AdamW(model.parameters(), lr=0.01)
        # Mini-batch training
        batch_size = 4096
        elapsed_time = []
        for epoch in range(1000):
            for i in range(0, X.shape[0], batch_size):
                x = X[i:i+batch_size]
                y = Y[i:i+batch_size]
                y_pred = model(x)
                loss = loss_fn(y_pred, y)
                loss.backward()
                opt.step()
                opt.zero_grad()
        torch.save(model.state_dict(), 'trained_models/{}_{}_{}'.format(context, emb_size, hidden_layer_size))

emb_size 2
	hidden_layer_size 5


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)